## 分车型预测-使用LSTM建模预测

In [2]:
import os
import warnings
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import plot_model
from keras.models import Sequential
from keras.engine.topology import Layer
from keras.layers import Dense, Dropout
from keras.initializers import glorot_uniform
from keras.layers import LSTM, RNN, GRU, SimpleRNN, Bidirectional, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

np.random.seed(1904)
warnings.filterwarnings('ignore')

C:\ProgramFiles\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framew

### 导入数据

In [3]:
data_path = './data/'
train_user_reply_data = pd.read_csv(data_path + 'train_user_reply_data.csv', encoding='utf-8')
train_search_data = pd.read_csv(data_path + 'train_search_data.csv', encoding='utf-8')
train_sales_data = pd.read_csv(data_path + 'train_sales_data.csv', encoding='utf-8')
evaluation_public = pd.read_csv(data_path + 'evaluation_public.csv', encoding='utf-8')

### 提取车型和省份数据

In [4]:
# cars = list(set(train_sales_data['model']))
# provinces = list(set(train_sales_data['province']))
# print(len(cars))
# print(len(provinces))
provinces = ['北京', '湖南', '江西', '山西', '河南', '四川', '山东', '湖北', '云南', '福建', '安徽', '河北', '浙江', '内蒙古', '陕西', '江苏', '重庆', '上海', '广东', '黑龙江', '辽宁', '广西']
cars = ['63065128401bb3ff', '02aab221aabc03b9', 'fde95ea242abd896', '3c974920a76ac9c1', '346393c2c6305fb1', '54fc07138d70374c', 'af6f4f548684e14d', 'bb9fbec9a2833839', 'feabbf46658382b9', 'a9a43d1a7ecbe75d', '3d7554f1f56dd664', 'd4efbebb087fd03f', 'a28bb927b6fcb33c', 'c6cd4e0e073f5ac2', '7023efdab9cedc03', 'b25c4e2e3856af22', '6858d6dfe680bdf7', '61e73e32ad101892', 'a432c483b5beb856', '3e21824be728cbec', 'fc32b1a017b34efe', 'ea489c253676aafc', '79de4e4b24c35b04', '7245e0ee27b195cd', 'd0f245b8781e3631', '8c915fe4632fb9fa', '0797526c057dcf5b', '5b1c11c3efed5312', '12f8b7e14947c34d', '7cf283430b3b5e38', '7aab7fca2470987e', 'a207df29ec9583f0', 'dff803b4024d261d', '936168bd4850913d', '2d0d2c3403909fdb', '212083a9246d2fd3', '4f79773e600518a6', '2a2ab41f8f6ff1cb', '37aa9169b575ef79', 'c6833cb891626c17', 'c06a2a387c0ee510', 'f270f6a489c6a9d7', '17bc272c93f19d56', '7a7885e2d7c00bcf', '04e66e578f653ab9', 'ef76a85c4b39f693', '17363f08d683d52b', '6155b214590c66e6', '5d7fb682edd0f937', 'f8a6975573af1b33', '4a103c30d593fbbe', 'b4be3a4917289c82', '28e29f2c03dcd84c', 'da457d15788fe8ee', '97f15de12cfabbd5', 'cd5841d44fd7625e', '9c1c7ee8ebdda299', 'cc21c7e91a3b5a0c', 'f5d69960089c3614', '06880909932890ca']

# Normalization
train_sales_data['salesVolume'] = train_sales_data['salesVolume'].map(lambda index: np.log2(index) + 1)
scaler = MinMaxScaler(feature_range=(0, 1))
train_sales_data[['salesVolume']] = scaler.fit_transform(train_sales_data[['salesVolume']])
print(train_sales_data[:3])

  province  adcode             model bodyType  regYear  regMonth  salesVolume
0       上海  310000  3c974920a76ac9c1      SUV     2016         1     0.557228
1       云南  530000  3c974920a76ac9c1      SUV     2016         1     0.609492
2      内蒙古  150000  3c974920a76ac9c1      SUV     2016         1     0.542952


### 构造数据

In [5]:
def generate_batch(data_set, look_back, gap_days):
    data_x, data_y = [], []
    for i in range(len(data_set) - look_back - gap_days + 1):
        a = data_set[i:(i + look_back), 0]
        data_x.append(a)
        data_y.append([data_set[i + look_back + gap_days - 1, 0]])
    return np.array(data_x), np.array(data_y), data_set[-look_back:, 0].reshape(1, look_back)

def generate_first_order_batch(data_set, look_back, gap_days):
    data_x = []
    for i in range(len(data_set) - look_back - gap_days + 1):
        a = data_set[i:(i + look_back), 0]
        data_x.append(a)
    return np.array(data_x), data_set[-look_back:, 0].reshape(1, look_back)

### 自定义评测指标

In [6]:
def metrics(data):
    data = scaler.inverse_transform(data)
    data = 2 ** (data - 1)
    print(data)
    
    a = data[:, 0]
    b = data[:, 1]
    res = np.sqrt(np.sum((a - b) ** 2) / len(a)) / np.mean(a)
    print('validate rmse:', 1 - res)
    return (1 - res)

### Adamw优化器&循环学习率

In [7]:
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates


class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        

    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

### 双输入-LSTM时序模型

In [8]:
from keras.layers.advanced_activations import PReLU

def construct_lstm_model(x_train, x_test, y_train, y_test, test, car_name, batch_size):
    print('batch_size:', batch_size)

    input1_ = Input(shape=(1, look_back), name='input1', dtype='float32')
    input2_ = Input(shape=(1, look_back), name='input2', dtype='float32')

    seq1 = LSTM(32, return_sequences=True)(input1_)    # , input_shape=(1, look_back)   GRU   LSTM
    seq1 = LSTM(32, return_sequences=False)(seq1)

    seq2 = LSTM(32, return_sequences=True)(input2_)
    seq2 = LSTM(32, return_sequences=False)(seq2)

    merged = concatenate([seq1, seq2])
    merged = BatchNormalization()(merged)
    merged = Dropout(0.5)(merged)

    output_ = Dense(1)(merged)
    output_ = PReLU()(output_)
    model = Model(inputs=[input1_, input2_], outputs=[output_])
    model.compile(loss='mean_squared_error', optimizer='adam')              # AdamW(weight_decay=0.08)   'adam'
#     model.summary()
    early_stopping = EarlyStopping('val_loss', patience=20, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=0)
    best_model_path = './model/' + car_name + '.h5'
    checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', mode='min', save_best_only=True, verbose=0, save_weights_only=False)
    
#     clr = CyclicLR(base_lr=0.001, max_lr=0.01, step_size=300., mode='exp_range', gamma=0.99994)
    clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
    clr = CyclicLR(base_lr=0.0001, max_lr=0.001, step_size=300., scale_fn=clr_fn, scale_mode='cycle')
    
    callbacks = [early_stopping, reduce_lr]         # checkpoint   # reduce_lr
    model.fit([x_train[:, :, :look_back], x_train[:, :, look_back:]], y_train, epochs=500, batch_size=batch_size, verbose=0, shuffle=False, callbacks=callbacks, validation_data=([x_test[:, :, :look_back], x_test[:, :, look_back:]], y_test))
#     model.load_weights(best_model_path)     # load best model parameters   shuffle=False
    predict = model.predict([test[:, :, :look_back], test[:, :, look_back:]])
    predict = scaler.inverse_transform(predict)
    predict = 2 ** (predict - 1)
#     print(np.hstack((y_test, model.predict(x_test))))
    rmse = metrics(np.hstack((y_test, model.predict([x_test[:, :, :look_back], x_test[:, :, look_back:]]))))
    return predict, rmse

### 分车型&分月建模预测

In [9]:
file_name = os.listdir('./model/')

look_back = 9     # 10  9
label_length = 4
batch_size = 16
submit = pd.DataFrame()
flag = 0
total = []

for car in cars:
    labels = np.zeros([len(provinces), label_length])
    rmse_total = 0
    for k in range(1, 5, 1):
        print('{} car:{}'.format(flag+1, car))
        train = np.array([[0 for _ in range(look_back)]])
        train_feature = np.array([[0 for _ in range(look_back)]])
        label = np.array([[0]])
        test = np.array([[0 for _ in range(look_back)]])
        test_feature = np.array([[0 for _ in range(look_back)]])

        for province in provinces: 
            temp = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
    #         print(temp[['province', 'model', 'regYear', 'regMonth', 'salesVolume']])
            temp['first_order'] = temp['salesVolume'].diff(1)

            data = temp['salesVolume'].values.reshape(-1, 1).astype('float32')[1:]
            data_b = temp['first_order'].values.reshape(-1, 1).astype('float32')[1:]

            train_data, train_label, test_data = generate_batch(data, look_back, k)
            train_data_b, test_data_b = generate_first_order_batch(data_b, look_back, k)

            train = np.vstack((train, train_data))
            label = np.vstack((label, train_label))
            test = np.vstack((test, test_data))
            train_feature = np.vstack((train_feature, train_data_b))
            test_feature = np.vstack((test_feature, test_data_b))

        train = np.hstack((train, train_feature))
        test = np.hstack((test, test_feature))
        train = train[1:]
        label = label[1:]
        test = test[1:]
        train = np.reshape(train, (-1, 1, look_back * 2))
        test = np.reshape(test, (-1, 1, look_back * 2))
        x_train,x_test,y_train,y_test = train_test_split(train, label, test_size=0.1, random_state=1904)

        print('train shape:', train.shape)
        print('label shape:', label.shape)
        print('test shape:', test.shape)
        res, rmse = construct_lstm_model(x_train, x_test, y_train, y_test, test, car, int((train.shape[0] * 0.9) / 22))
        
        K.clear_session()    #  清除tf session

        labels[:, k-1] = res.reshape(1, -1)
        rmse_total += rmse
    print(labels)
    print('average rmse:', rmse_total / 4)
    flag += 1
    total.append(rmse_total / 4)

    index = 0
    submit_partial = pd.DataFrame()
    for province_2 in provinces: 
        temp = evaluation_public[(evaluation_public['model'] == car) & (evaluation_public['province'] == province_2)]
        temp['forecastVolum'] = labels[index]
        submit_partial = pd.concat([submit_partial, temp], axis=0)
        index += 1
    submit = pd.concat([submit, submit_partial], axis=0)

print('all validation rmse:', np.mean(total))  # 0.6555167883490066
car_rmse = pd.DataFrame({'cars': cars, 'rmse': total})
print(car_rmse)
# car_rmse.to_csv('car_rmse.csv', encoding='utf-8', index=None)

[]
1 car:63065128401bb3ff
train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00031: early stopping
[[ 691.99995217 1933.38750864]
 [ 213.99998516  384.05053383]
 [2187.99985726 2094.61830788]
 [1218.00022926 1251.88118693]
 [1543.99992576 1804.32234767]
 [2400.99986102 2363.8903343 ]
 [ 861.00018704 1010.99140879]
 [1223.00025475 1638.30938908]
 [3813.99934853 2938.9302254 ]
 [1033.00008916 1033.06507027]
 [1030.99979211  752.03382252]
 [1307.00008176 1450.9150229 ]
 [4090.99959642 3164.60211109]
 [3307.00016327 3355.5433662 ]
 [1550.00023406  970.44302813]
 [4744.99898154 4027.92425944]
 [1398.99963257 1745.97256651]
 [2429.00015662 2313.69521389]
 [2449.99960778 2884.12198502]
 [1328.00003096 1601.07419807]
 [2268.99946119 2012.72733164]
 [ 972.99989301 2055.39241437]
 [ 821.99996833  648.09962745]
 [4376.99941314 3582.16366546]
 [ 923.00021537  406.66986251]
 [ 216.99998578 1357.69702151]
 [1474.00019371 1895.39881674]
 [1980.00015659 1432.7

Epoch 00066: early stopping
[[ 251.99997215  284.84079005]
 [ 669.99992742  744.88203936]
 [ 277.99995496  244.17446656]
 [ 547.99987753  523.56051206]
 [ 574.99993604  605.45222007]
 [ 346.00006825  267.26063032]
 [1428.00031926 1260.9128361 ]
 [ 137.00000586  144.77363356]
 [ 972.99989301  950.60647341]
 [ 335.00005295  293.99389454]
 [ 646.99983716  669.57789433]
 [ 254.99994193  247.46037559]
 [ 240.0000128   227.50913791]
 [ 275.00003008  265.72041638]
 [1360.99992584 1351.70453333]
 [ 544.00013778  543.75166906]
 [ 215.9999473   178.2691066 ]
 [1536.99996726 1339.04764118]
 [ 346.00006825  297.56627031]
 [ 203.99998059  226.43217985]
 [ 261.00001421  225.29557648]
 [ 627.00007424  532.00163685]
 [ 476.99993557  431.01870943]
 [ 164.99998246  133.68980997]
 [ 339.00003597  322.0977506 ]]
validate rmse: 0.8764571798971973
[[ 640.71533203  564.44598389  645.53973389  590.72338867]
 [ 282.45648193  273.07580566  295.69125366  278.11950684]
 [ 242.72335815  200.9198761   247.14892578 

Epoch 00077: early stopping
[[ 193.9999825   289.75193223]
 [ 798.99993297  676.48179251]
 [ 624.00011502  560.16536209]
 [ 175.99997209  174.58899632]
 [ 308.00004924  264.37560801]
 [ 209.00004177  281.66627623]
 [ 335.99999034  397.32101793]
 [ 449.0001188   357.06231618]
 [ 432.00000981  321.79260668]
 [ 280.00005231  288.94107012]
 [ 167.99999514  162.84769542]
 [ 297.00003849  272.63829735]
 [ 449.0001188   384.42168208]
 [ 145.99998818  190.4808422 ]
 [ 601.00014728  407.87198493]
 [ 297.00003849  226.94140712]
 [ 616.99999424  637.45984078]
 [2695.99958708 2351.05425293]
 [ 260.00003463  208.99837057]
 [ 275.00003008  324.38364045]
 [2958.00036127 1917.65309392]
 [ 235.99993931  135.22196533]
 [ 303.00000948  304.476846  ]
 [ 676.00017861  536.65387606]
 [ 308.99993282  243.20420351]
 [ 144.000015    156.20348807]
 [ 292.99992504  395.51949182]
 [ 310.99999849  230.60765462]
 [ 218.99996439  187.58298892]]
validate rmse: 0.5818793950825417
4 car:3c974920a76ac9c1
train shape: (2

train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00032: early stopping
[[ 213.00003262  171.45971402]
 [ 129.00000071  211.54110703]
 [1835.99952625 1117.61277495]
 [ 328.99999414  244.60438624]
 [1066.99973392  706.64612039]
 [ 623.00000063  437.01000811]
 [ 274.00001175  290.86292865]
 [ 139.00001451  141.69756732]
 [ 251.99997215  168.63350498]
 [ 485.00002132  314.7973096 ]
 [ 132.99998377  153.76160288]
 [ 299.00004546  311.72599857]
 [ 495.00003899  416.23291356]
 [ 545.00008749  401.36615646]
 [ 398.00000325  365.96608895]
 [ 439.99998925  459.46305504]
 [ 507.99999809  470.32096507]
 [ 614.99985624  500.60561197]
 [ 849.00009697  580.85310064]
 [ 341.0000874   279.81338782]
 [ 546.00012889  494.37373521]
 [ 297.00003849  221.02780161]
 [ 238.00000912  255.84917927]
 [ 420.99990696  447.99739673]
 [ 419.0000991   321.3446829 ]
 [ 533.99992926  486.46532542]
 [ 325.00000021  274.59703847]
 [ 185.00002976  122.72893082]
 [ 595.00010261  

Epoch 00037: early stopping
[[ 311.9999743   268.01522201]
 [ 503.00000204  460.12337003]
 [1084.00020992 1042.74638313]
 [1018.99977723 1231.2746993 ]
 [ 489.00005739  504.82692125]
 [1068.00014336  878.20301746]
 [1406.99999231 1429.93897997]
 [ 257.00000759  250.61436096]
 [ 537.00014281  566.68135715]
 [1019.99976803  970.37888229]
 [ 574.00001853  706.84466801]
 [ 413.00002531  304.3270707 ]
 [ 321.0000371   273.27824193]
 [ 893.00002009  826.56892342]
 [1023.99972849  897.19804811]
 [1018.00022413 1017.95626849]
 [ 745.99986478  658.4973865 ]
 [ 986.00020072  980.80508651]
 [ 923.00021537  748.12488701]
 [ 432.99993244  359.10173447]
 [ 311.9999743   302.47100021]
 [ 869.9997714   801.66344112]
 [ 398.00000325  409.51992858]
 [ 452.0000849   442.40980968]
 [1014.99986878  958.66529894]]
validate rmse: 0.8778918953094831
[[ 410.92861938  520.31829834  462.84655762  413.18505859]
 [ 974.03881836  965.16375732  875.09008789  842.24645996]
 [ 903.41699219  768.13220215  659.39788818 

...

Epoch 00051: early stopping
[[ 89.99999743  64.15612199]
 [ 88.00000949 152.46891522]
 [ 74.99999403  57.25043928]
 [149.00001603 179.71945201]
 [137.00000586 202.2677356 ]
 [ 60.99999469  56.61903027]
 [487.00008622 298.9633879 ]
 [ 30.00000159  23.03893384]
 [289.99994741 259.53669104]
 [ 94.99999621  93.81312117]
 [148.00000398 146.8835636 ]
 [ 74.99999403  62.67724137]
 [ 33.99999953  31.64249544]
 [ 72.00000749  46.74409835]
 [231.00001805 196.69224081]
 [153.99998355 139.75358786]
 [ 39.99999472  38.82678559]
 [511.00012141 329.12945211]
 [139.99998882 114.87840892]
 [ 64.00000005  52.87262005]
 [ 32.00000002  28.94952165]
 [218.99996439 137.26529458]
 [ 64.00000005  72.27374631]
 [ 47.99999611  30.18853365]
 [ 66.00000172  78.94498769]]
validate rmse: 0.5589449200588954
[[188.48597717 103.19406128 109.27294159 111.9598465 ]
 [ 82.44002533  56.12828445  48.52261734  62.86833954]
 [ 55.86650467  36.91265869  32.88195419  36.71067429]
 [ 32.15263367  35.09800339  19.42809105  18.76

Epoch 00046: early stopping
[[ 166.00000379  283.59643382]
 [1308.00027686 1038.87863513]
 [1317.00003868  945.37960392]
 [ 193.9999825   196.20559479]
 [ 482.99993117  386.0384619 ]
 [ 193.9999825   275.54150383]
 [ 546.99996414  453.82462784]
 [ 959.00009105  528.68089526]
 [ 423.99989897  821.76556987]
 [ 420.99990696  416.22181947]
 [ 255.99993204  247.75960944]
 [ 342.00006709  377.33605294]
 [ 541.0000357   493.27809778]
 [  44.00000474  124.1144934 ]
 [ 646.00005078  530.32170448]
 [ 337.00003805  379.82439753]
 [ 932.99991951  937.52291648]
 [1202.99997605  782.51010413]
 [ 103.00001345  153.94810884]
 [1125.99983606  725.2183609 ]
 [1005.99973609 1089.97606162]
 [ 281.99995734  200.99982769]
 [ 435.00000158  495.10797515]
 [1678.00021347 1072.71773439]
 [ 455.99988364  497.37239015]
 [ 264.00000696  268.13469056]
 [ 919.99977455  690.53001249]
 [ 411.99994411  391.15693231]
 [ 199.99994716  198.26456719]]
validate rmse: 0.6299835182690063
54 car:da457d15788fe8ee
train shape: (

train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00044: early stopping
[[ 425.99995176  369.66855074]
 [ 318.99993356  420.38408413]
 [ 995.00014158  581.31309844]
 [ 325.00000021  278.19831163]
 [ 804.00016818  767.95347233]
 [ 746.99989529  528.26648823]
 [ 765.0001713   819.63976409]
 [ 314.99992343  350.93676966]
 [ 753.99996206  610.37377269]
 [ 442.00011112  349.67888004]
 [ 314.99992343  298.34725818]
 [ 417.99997219  425.10048539]
 [1805.99962308 1468.94937354]
 [1241.00021283 1094.4995428 ]
 [1047.00018148 1153.16228854]
 [1162.00008721 1100.14086239]
 [ 852.00013075  888.0671733 ]
 [1324.99968636 1045.14491808]
 [1082.99982332  678.61857519]
 [ 379.00000058  413.2435956 ]
 [1694.00000113 1365.9584317 ]
 [ 364.00001861  251.36000466]
 [ 469.99990502  487.33042114]
 [1624.99978615 1556.81289887]
 [ 995.9999418   998.34895808]
 [1263.00029131 1226.41016064]
 [ 646.00005078  679.19690902]
 [ 517.99990282  377.45635964]
 [1577.99962826 1

Epoch 00036: early stopping
[[ 122.99998751   76.8946346 ]
 [  10.99999971   59.5726177 ]
 [  50.0000001   125.21052686]
 [ 649.99982727  506.28276121]
 [  12.99999998  144.12941926]
 [  86.99998863  150.42899551]
 [ 351.00000732  177.827189  ]
 [ 134.0000032    83.29835334]
 [ 197.00001339   81.17601247]
 [ 153.00001371   54.11888949]
 [  41.99999877   90.1384937 ]
 [ 775.99993026  281.53687649]
 [  25.00000005  118.95833417]
 [  54.99999863   52.95075087]
 [ 470.9998974   337.47590048]
 [ 454.00005671  427.90061931]
 [  41.99999877  228.01203552]
 [ 222.99998655   62.63705976]
 [1481.99961955  784.21388435]
 [ 274.00001175  140.70365556]
 [  80.99999519   75.53658874]
 [  22.00000237   31.70029553]
 [  57.99999715   80.24746391]
 [ 461.00010567  184.53149103]
 [  38.99999677  110.56213125]]
validate rmse: 0.1915540208459544
[[ 140.8269043    54.72396088   19.65753937    8.38460255]
 [ 452.50942993  206.93449402   63.01054764   25.36350822]
 [ 320.26980591  165.10749817   64.34998322 

Epoch 00035: early stopping
[[ 47.99999611  43.01099642]
 [212.00000596 177.98499925]
 [109.99999728 108.23901954]
 [ 72.00000749  53.94599646]
 [157.00002043 124.44030873]
 [ 70.99999773  79.14442212]
 [231.00001805 167.21837295]
 [444.00009413 187.65219922]
 [232.99994078 377.76635616]
 [134.99998513 123.88759739]
 [ 67.00000159  63.75628976]
 [132.00000346 104.24664022]
 [193.00004207 181.86311667]
 [ 20.99999938  33.62499026]
 [356.00007678 154.47357207]
 [112.0000059   83.41268465]
 [177.00003437 153.22332743]
 [497.00001006 166.05762144]
 [ 50.99999513  30.6738759 ]
 [289.99994741 292.3487047 ]
 [309.99992256 233.66676815]
 [ 83.99999756  58.67327695]
 [241.00001318 185.54946442]
 [126.99999948 124.1049994 ]
 [251.99997215 166.29392113]
 [ 80.99999519  87.84175728]
 [577.99998394 341.16336472]
 [112.99999107  83.75804975]
 [ 64.00000005  45.75937055]]
validate rmse: 0.4413340360907759
59 car:f5d69960089c3614
train shape: (264, 1, 18)
label shape: (264, 1)
test shape: (22, 1, 18)


```
all validation rmse: 0.6182310141381996 

```

In [10]:
submit.sort_values(by=['id'], ascending=True)['forecastVolum'].mean()

568.2767967775012

In [11]:
submit = submit.sort_values(by=['id'], ascending=True)
submit['forecastVolum'] = submit['forecastVolum'].map(lambda index: int(np.round(index)))
print('res store over')

res store over


In [12]:
thresh = submit['forecastVolum'].mean() / 450
submit.groupby(['regMonth'])['forecastVolum'].mean()

regMonth
1    745.329545
2    534.420455
3    487.915152
4    505.452273
Name: forecastVolum, dtype: float64

In [13]:
sub = submit.copy()
sub['forecastVolum'] = sub['forecastVolum'] / thresh
sub['forecastVolum'] = sub['forecastVolum'].map(lambda index: int(np.round(index)))
sub.groupby(['regMonth'])['forecastVolum'].mean()
sub[['id', 'forecastVolum']].to_csv('./submit/chusai_lstm_model_2.csv', encoding='utf-8', index=None)

regMonth
1    590.193182
2    423.180303
3    386.362879
4    400.250758
Name: forecastVolum, dtype: float64